In [1]:
# cr： /lstm-model-of-stockdata/
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/water3/cleaned_train.csv
/kaggle/input/water3/cleaned_tests.csv


In [35]:
import matplotlib.pyplot as plt
import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt
from sklearn import linear_model 
from sklearn.metrics import mean_absolute_error
import plotly

# import the relevant Keras modules
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
from datetime import datetime 

# print(check_output(["ls", "../input"]).decode("utf8"))

In [45]:
train = pd.read_csv('../input/water3/cleaned_train.csv')
tests = pd.read_csv('../input/water3/cleaned_tests.csv')
cols = ['year', 'month', 'date']
train['ymd'] = train[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
train['ymd'] = pd.to_datetime(train['ymd'])
train.info()
tests['ymd'] = tests[cols].apply(lambda row: '-'.join(row.values.astype(str)), axis=1)
tests['ymd'] = pd.to_datetime(tests['ymd'])
df = train.drop(['sub','div','log'],axis = 1)
window_len = 7

#Create a data point (i.e. a date) which splits the training and testing set
split_date = list(df["ymd"][-(10*window_len+1):])[0]

#Split the training and test set
training_set, test_set = df[df['ymd'] < split_date], df[df['ymd'] >= split_date]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1032 entries, 0 to 1031
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   a_fc    1032 non-null   float64       
 1   b_fc    1032 non-null   float64       
 2   year    1032 non-null   int64         
 3   month   1032 non-null   int64         
 4   date    1032 non-null   int64         
 5   sum     1032 non-null   float64       
 6   sub     1032 non-null   float64       
 7   div     1032 non-null   float64       
 8   log     1032 non-null   float64       
 9   sub_1a  1032 non-null   float64       
 10  sub_1b  1032 non-null   float64       
 11  sub_2a  1032 non-null   float64       
 12  sub_2b  1032 non-null   float64       
 13  sub_3a  1032 non-null   float64       
 14  sub_3b  1032 non-null   float64       
 15  ymd     1032 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(12), int64(3)
memory usage: 129.1 KB


In [46]:
training_set = training_set.drop(['ymd'], 1)
test_set = test_set.drop(['ymd'], 1)

tests_set = tests.drop(['sub','div','log'],axis = 1)
tests_set = tests_set.drop(['ymd'], 1)
tests_set = tests_set.rename(columns = {'A厂':'a_fc','B厂':'b_fc'})

# 1st part predict pseudo out

In [62]:
training_set_1 = training_set.drop(['a_fc','b_fc'], axis = 1)
test_set_1 = test_set.drop(['a_fc','b_fc'], axis = 1)
tests_set_1 = tests_set.drop(['a_fc','b_fc'], axis = 1)

In [64]:
training_set_1.head(3)

,year,month,date,sum,sub_1a,sub_1b,sub_2a,sub_2b,sub_3a,sub_3b
0,2018,1,1,390182.0,17673.0,21657.0,23809.0,28277.0,15739.0,23146.0
1,2018,1,2,429512.0,6136.0,6620.0,-1934.0,1489.0,-17918.0,-1720.0
2,2018,1,3,442268.0,-8070.0,-5131.0,-24054.0,-8340.0,-26665.0,-31540.0


In [47]:
#Create windows for training
LSTM_training_inputs = []
for i in range(len(training_set)-window_len):
    temp_set = training_set[i:(i+window_len)].copy()
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_training_inputs.append(temp_set)
    

LSTM_training_outputs1 = (training_set['sum'][window_len:].values/training_set['sum'][:-window_len].values)-1
LSTM_training_outputs2 = (training_set['sub_1a'][window_len:].values/training_set['sub_1a'][:-window_len].values)-1
LSTM_training_outputs3 = (training_set['sub_1b'][window_len:].values/training_set['sub_1b'][:-window_len].values)-1
LSTM_training_outputs4 = (training_set['sub_2a'][window_len:].values/training_set['sub_2a'][:-window_len].values)-1
LSTM_training_outputs5 = (training_set['sub_2b'][window_len:].values/training_set['sub_2b'][:-window_len].values)-1
LSTM_training_outputs6 = (training_set['sub_3a'][window_len:].values/training_set['sub_3a'][:-window_len].values)-1
LSTM_training_outputs7 = (training_set['sub_3b'][window_len:].values/training_set['sub_3b'][:-window_len].values)-1

LSTM_training_inputs = [np.array(LSTM_training_input) for LSTM_training_input in LSTM_training_inputs]
LSTM_training_inputs = np.array(LSTM_training_inputs)

#Create windows for testing
LSTM_test_inputs = []
for i in range(len(test_set)-window_len):
    temp_set = test_set[i:(i+window_len)].copy()
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_test_inputs.append(temp_set)
    

LSTM_test_outputs1 = (test_set['a_fc'][window_len:].values/test_set['a_fc'][:-window_len].values)-1
LSTM_test_outputs2 = (test_set['b_fc'][window_len:].values/test_set['b_fc'][:-window_len].values)-1

LSTM_test_inputs = [np.array(LSTM_test_inputs) for LSTM_test_inputs in LSTM_test_inputs]
LSTM_test_inputs = np.array(LSTM_test_inputs)

In [48]:
# LSTM_training_outputs1
# LSTM_training_inputs.shape
LSTM_test_inputs.shape

(64, 7, 12)

# undone following part
6.11

In [52]:
#Create windows for testing
LSTM_tests_inputs = []
for i in range(len(tests_set)-window_len):
    temp_set = tests_set[i:(i+window_len)].copy()
    for col in list(temp_set):
        temp_set[col] = temp_set[col]/temp_set[col].iloc[0] - 1
    LSTM_tests_inputs.append(temp_set)
    


In [58]:
LSTM_tests_outputs1 = (tests_set['a_fc'][window_len:].values/tests_set['a_fc'][:-window_len].values)-1
LSTM_tests_outputs2 = (tests_set['b_fc'][window_len:].values/tests_set['b_fc'][:-window_len].values)-1

LSTM_tests_inputs = [np.array(LSTM_tests_inputs) for LSTM_tests_inputs in LSTM_tests_inputs]
LSTM_tests_inputs = np.array(LSTM_tests_inputs)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in true_divide
  


In [ ]:
LSTM_training_outputs = pd.DataFrame({'col1': LSTM_training_outputs1,'col2':LSTM_training_outputs2})
LSTM_test_outputs = pd.DataFrame({'col1': LSTM_test_outputs1,'col2':LSTM_test_outputs2})


---
model

In [ ]:
def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.10, loss="mae", optimizer="adam"):
    
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model


# bdp入湖开发培训
# jiangyuqing@sfmail.sf-express.com
# elenore@sfmail.sf-express.com

# initialise model architecture
nn_model = build_model(LSTM_training_inputs, output_size=2, neurons = 328)
# model output is next price normalised to 10th previous closing price
# train model on data
# note: eth_history contains information on the training error per epoch
nn_history = nn_model.fit(LSTM_training_inputs, LSTM_training_outputs, 
                            epochs=25, batch_size=10, verbose=2, shuffle=True)

In [ ]:
MAE = mean_absolute_error(LSTM_test_outputs, nn_model.predict(LSTM_test_inputs))
print('The Mean Absolute Error is: {}'.format(MAE))

# ad hoc model, simply predict with time;. 

In [ ]:
train = train[['year','month','date','ymd']]
tests = tests[['year','month','date','ymd']]

In [ ]:
LSTM_training_outputs

In [ ]:

# initialise model architecture
nn_model = build_model(LSTM_training_inputs, output_size=2, neurons = 328)
# model output is next price normalised to 10th previous closing price
# train model on data
# note: eth_history contains information on the training error per epoch
nn_history = nn_model.fit(LSTM_training_inputs, LSTM_training_outputs, 
                            epochs=25, batch_size=10, verbose=2, shuffle=True)